In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=984d012c8d286f3052e79fc9f8650952e69bdbc1d4124c5ae55c079a3da69bcc
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [4]:
from efficientnet_pytorch import EfficientNet
import torch
import torch.nn as nn
from PIL import Image
import os
from torchvision import transforms
import shutil


In [6]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
  transforms.Resize((224, 224)),  # Resize images to 224x224 for EfficientNet
  transforms.ToTensor(),  # Convert to tensor
  transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])

def classify_images(model_path, image_dir, output_good_dir, output_bad_dir):
  # Load the model
  model = EfficientNet.from_pretrained('efficientnet-b0')
  num_features = model._fc.in_features
  model._fc = nn.Linear(num_features, 1)
  weights = torch.load(model_path, map_location=device)  # Load weights only
  model.load_state_dict(weights, strict=True)  # Apply weights to model
  model = model.to(device)
  model.eval()

  # Process each image in the directory
  for root, _, files in os.walk(image_dir):
    for file in files:
      image_path = os.path.join(root, file)
      if not file.lower().endswith('.tif'):
        continue  # Skip non-TIF files
      try:
        # Load and preprocess the image
        image = Image.open(image_path).convert("RGB")  # Open and convert to RGB
        transformed_image = transform(image).unsqueeze(0).to(device)  # Apply transformations and add batch dimension

        # Predict
        with torch.no_grad():
          output = model(transformed_image)
          prediction = torch.sigmoid(output).item()

        # Classify and move to appropriate folder
        if prediction > 0.5:
          shutil.copy(image_path, os.path.join(output_good_dir, file))
        else:
          shutil.copy(image_path, os.path.join(output_bad_dir, file))

      except Exception as e:
        print(f"Error processing {image_path}: {e}")

# all paths(directories) must be created/downloaded before running the classifier

model_path = "/content/drive/MyDrive/IBCh/LIveMIELmiiish2/untresh2/binary_classifier2.pth" # path where is model
image_dir =  "/content/drive/MyDrive/IBCh/LIveMIELmiiish2/untresh2/test_test"              # path where are all images
output_good_dir = "/content/drive/MyDrive/IBCh/LIveMIELmiiish2/untresh2/test_test_good"    # path where will be "good" images
output_bad_dir = "/content/drive/MyDrive/IBCh/LIveMIELmiiish2/untresh2/test_test_bad"      # path where will be "bad" images

classify_images(model_path, image_dir, output_good_dir, output_bad_dir)

Loaded pretrained weights for efficientnet-b0


<ipython-input-6-5d7cfea3d975>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(model_path, map_location=device)  # Load weights only
